## Import Packages

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

C:\Users\Neelotpal\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Import Data

In [3]:
data = pd.read_csv('pay-discrimination.csv')
data.head()

,Unnamed: 0,female,cg,sc,hsg,mw,so,we,ne,exp1,exp2,exp3,wage
0,1663473,0,0,0,1,0,0,0,1,33.0,10.89,35.937,11.659091
1,1663483,0,1,0,0,0,0,0,1,27.0,7.29,19.683,12.825000
2,1663490,0,0,1,0,0,0,0,1,13.0,1.69,2.197,5.777027
3,1663497,0,1,0,0,0,0,0,1,2.0,0.04,0.008,12.468750
4,1663500,1,1,0,0,0,0,0,1,15.0,2.25,3.375,18.525000


## Preprocess and Examine Data

In [4]:
del data['Unnamed: 0']

In [5]:
data.head()

,female,cg,sc,hsg,mw,so,we,ne,exp1,exp2,exp3,wage
0,0,0,0,1,0,0,0,1,33.0,10.89,35.937,11.659091
1,0,1,0,0,0,0,0,1,27.0,7.29,19.683,12.825000
2,0,0,1,0,0,0,0,1,13.0,1.69,2.197,5.777027
3,0,1,0,0,0,0,0,1,2.0,0.04,0.008,12.468750
4,1,1,0,0,0,0,0,1,15.0,2.25,3.375,18.525000


In [24]:
data.groupby('female').mean()

,cg,sc,hsg,mw,so,we,ne,exp1,exp2,exp3,wage
female,,,,,,,,,,,
0,0.354839,0.301971,0.343190,0.284946,0.235215,0.221326,0.258513,13.580197,2.586588,5.964938,16.117458
1,0.406114,0.354336,0.239551,0.291329,0.255147,0.198378,0.255147,13.037118,2.449453,5.599297,14.720058


## Perform Ordinary Least Sqaures Regression 
### Basic Model

In [17]:
pred = data.iloc[:,0:11]
targ = data.iloc[:,11]

,female,cg,sc,hsg,mw,so,we,ne,exp1,exp2,exp3
0,0,0,0,1,0,0,0,1,33.0,10.89,35.937
1,0,1,0,0,0,0,0,1,27.0,7.29,19.683
2,0,0,1,0,0,0,0,1,13.0,1.69,2.197
3,0,1,0,0,0,0,0,1,2.0,0.04,0.008
4,1,1,0,0,0,0,0,1,15.0,2.25,3.375


In [24]:
pred = sm.add_constant(pred)
modelb1 = sm.OLS(targ, pred).fit()

In [79]:
resb1 = [modelb1.params[1], modelb1.bse[1]] + modelb1.conf_int(alpha=0.05, cols=None).iloc[1,:].values.tolist()
print(resb1)

[-1.8263969246850196, 0.42451632822255275, -2.658697006484514, -0.9940968428855252]


### Flexible Model

In [52]:
modelf1 = smf.ols('wage ~  female + (sc+ cg+ mw + so + we + exp1 + exp2 + exp3)**2', data=data).fit()

In [65]:
resf1 = [modelf1.params[1], modelf1.bse[1]] + modelf1.conf_int(alpha=0.05, cols=None).iloc[1,:].values.tolist()
print(resf1)

[-1.8800128627319634, 0.42474381514234977, -2.7127605464142843, -1.0472651790496426]


### Present Results

In [84]:
res = pd.DataFrame([resb1])
res = res.append([resf1], ignore_index = True)

In [94]:
res.columns = ['Estimate', 'Std Err', '[0.025', '0.975]']
res.head()

,Estimate,Std Err,[0.025,0.975]
0,-1.826397,0.424516,-2.658697,-0.994097
1,-1.880013,0.424744,-2.712761,-1.047265


## Perform Partialling Out Regression
### Basic Model

In [6]:
mod_out = smf.ols('wage ~  sc+ cg+ mw + so + we + exp1 + exp2 + exp3', data=data).fit()
mod_treat = smf.ols('female ~  sc+ cg+ mw + so + we + exp1 + exp2 + exp3', data=data).fit()

In [9]:
modelb2 = sm.OLS(mod_out.resid, mod_treat.resid).fit()

In [10]:
modelb2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     18.55
Date:                Wed, 01 Nov 2017   Prob (F-statistic):           1.69e-05
Time:                        21:34:53   Log-Likelihood:                -15235.
No. Observations:                3835   AIC:                         3.047e+04
Df Residuals:                    3834   BIC:                         3.048e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -1.8264      0.424     -4.307      0.000      -2.658      -0.995
==============================================================================
Omnibus:                     6626.018   Durbin-Watson:                   1.958
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          8721375.158
Skew:                          11.808   Prob(JB):                         0.00
Kurtosis:                     235.426   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""